<a href="https://colab.research.google.com/github/pablolube/Data-analyst-projects/blob/main/Data_Cleaning_Usuarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

Web Scraping

In [ ]:
# prompt: url = {}
# usuarios={}
# for i in range(2015, 2025):
#     url[i] = "https://raw.githubusercontent.com/pablolube/Data-analyst-projects/refs/heads/main/Ecobicis_Analysis/Bases/usuarios-ecobici-" + str(i) + ".csv"
#     usuario = pd.read_csv(url[i])

import pandas as pd
import numpy as np

url = {}
usuarios={}
for i in range(2015, 2019):
    url[i] = "https://raw.githubusercontent.com/pablolube/Data-analyst-projects/refs/heads/main/Ecobicis_Analysis/Bases/usuarios-ecobici-" + str(i) + ".csv"
    usuarios[i] = pd.read_csv(url[i])
    print(f"Data for year {i} loaded successfully.") # Optional: Print confirmation for each year
    # Process each dataframe (usuarios[i]) as needed. For example:
    #print(usuarios[i].head())

for i in range(2019, 2025):
    url[i] = "https://raw.githubusercontent.com/pablolube/Data-analyst-projects/refs/heads/main/Ecobicis_Analysis/Bases/usuarios_ecobici_" + str(i) + ".csv"
    usuarios[i] = pd.read_csv(url[i])
    print(f"Data for year {i} loaded successfully.") # Optional: Print confirmation for each year
    # Process each dataframe (usuarios[i]) as needed. For example:
    #print(usuarios[i].head())

Data for year 2015 loaded successfully.
Data for year 2016 loaded successfully.
Data for year 2017 loaded successfully.
Data for year 2018 loaded successfully.
Data for year 2019 loaded successfully.
Data for year 2020 loaded successfully.
Data for year 2021 loaded successfully.
Data for year 2022 loaded successfully.


<ipython-input-52-803034f9b68e>:21: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  usuarios[i] = pd.read_csv(url[i])


Data for year 2023 loaded successfully.
Data for year 2024 loaded successfully.


Imprimo los dataset antes de hacer un merge

In [ ]:
for i in range(2015, 2025):
    print(f"\n{'='*40}")
    print(f"Usuarios del año {i}:")
    print(f"{'='*40}\n")
    print(usuarios[i].head().to_string(index=False))




Usuarios del año 2015:

 id_usuario genero_usuario  edad_usuario fecha_alta  hora_alta
       7682              M            45   28-02-15 5:32:55 PM
      19577              M            47   28-02-15 5:33:12 PM
     108635              M            26   28-02-15 5:35:14 PM
     129948              M            29   28-02-15 5:35:43 PM
     129949              M            24   28-02-15 5:35:43 PM

Usuarios del año 2016:

 id_usuario genero_usuario  edad_usuario fecha_alta   hora_alta
     223858              M            41   01-01-16  3:45:19 AM
     223873              M            35   01-01-16 12:36:42 PM
     223874              M            47   01-01-16 12:45:44 PM
     223875              F            42   01-01-16 12:54:44 PM
     223876              M            49   01-01-16 12:58:38 PM

Usuarios del año 2017:

 id_usuario genero_usuario  edad_usuario fecha_alta   hora_alta
     336112              M            21   01-01-17  1:32:11 AM
     336114              M         

En principio hay ciertas correcciones a hacer sobre todo la hora y el genero pero las misma no se pueden hacer despues del append, hay 2 columnas que notienen antes de

In [ ]:
# prompt: df_usuarios = usuarios[2015]  # Comienza con el primer DataFrame
# # Agregar los demás DataFrames usando append()
# for i in range(2016, 2025):
#     df_usuarios =pd.df_usuarios.append(usuarios[i], ignore_index=True)
# print(df_usuarios)

df_usuarios = usuarios[2024].copy()  # Comienza con el primer DataFrame, usando .copy() para evitar SettingWithCopyWarning

# Agregar los demás DataFrames usando concat() para mejor rendimiento
for i in range(2024,2015 ):
    df_usuarios = pd.merge([df_usuarios, usuarios[i]], ignore_index=True)

df_usuarios

,ID Cliente,genero_usuario,edad_usuario,fecha_alta,hora_alta,Customer Has Dni (Yes / No)
0,1262772,OTHER,22,2024-12-03,11:09:31,No
1,1262793,MALE,30,2024-12-03,11:32:23,No
2,1262901,MALE,28,2024-12-03,14:19:14,No
3,1262787,MALE,27,2024-12-03,11:26:06,No
4,1263203,FEMALE,51,2024-12-03,20:06:49,No
...,...,...,...,...,...,...
179718,1084451,OTHER,33,2024-01-01,22:25:31,No
179719,1084135,OTHER,40,2024-01-01,16:26:35,No
179720,1083797,FEMALE,19,2024-01-01,12:18:18,Yes
179721,1084368,OTHER,51,2024-01-01,18:59:58,No


from matplotlib import pyplot as plt
df_usuarios['ID Cliente'].plot(kind='hist', bins=20, title='ID Cliente')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
df_usuarios.groupby('genero_usuario').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
df_usuarios.groupby('Customer Has Dni (Yes / No)').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df_usuarios['ID Cliente'].plot(kind='line', figsize=(8, 4), title='ID Cliente')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['Customer Has Dni (Yes / No)'].value_counts()
    for x_label, grp in df_usuarios.groupby('genero_usuario')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('genero_usuario')
_ = plt.ylabel('Customer Has Dni (Yes / No)')

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df_usuarios['genero_usuario'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df_usuarios, x='ID Cliente', y='genero_usuario', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df_usuarios['Customer Has Dni (Yes / No)'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df_usuarios, x='ID Cliente', y='Customer Has Dni (Yes / No)', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)